### 🧪 Run AI Foundry agent operations with Business API Backend, Bing Search and Logic Apps workflow

⚙️ **Tools**:
- Get Product Catalog - OpenAPI Backend mocked with an APIM policy.
- Place Order - A Logic Apps workflow that processes orders with a maximum of five items.
- Weather - Weather API from Azure APIM
- Bing - Grounding with Bing Search

✨ **Expected Behavior**:
- The agent receives a user request to order 11 smartphones.
- The agent calls the product catalog API to retrieve the product SKU and available stock quantity.
- If the order quantity exceeds available stock, the agent will respond that the order cannot be processed due to insufficient stock.
- If stock is available, the agent will initiate the order workflow, which will fail because the quantity exceeds the maximum limit of five items.
- As the agent was instructed to recover from errors, it will place multiple orders, each with a quantity below the maximum limit, ensuring the total equals the desired order quantity.


In [ ]:
# The following code was adapted from this sample:
# https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py

import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme, ToolSet
from azure.ai.projects.models import BingGroundingTool
# from azure.monitor.opentelemetry import configure_azure_monitor

prompt_content = "Please order one smartphone for me and one for each of my ten friends."
# prompt_content = "Provide me with the weather in Seattle and 3 other sister cities in Europe."
# prompt_content = "What are the top news today?"
# prompt_content = "I need to solve the equation `3x + 11 = 14`. Can you help me?"

project_connection_string = "eastus2.api.azureml.ms;29e274a2-ee66-4823-8b20-63fafc234826;lab-ai-agent-service;project-57r5eue6ypi6s"
project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(),
    conn_str=project_connection_string)

# application_insights_connection_string = project_client.telemetry.get_connection_string()

apim_resource_gateway_url = "https://apim-57r5eue6ypi6s.azure-api.net"
product_catalog_api_connection_id = "/subscriptions/29e274a2-ee66-4823-8b20-63fafc234826/resourceGroups/lab-ai-agent-service/providers/Microsoft.MachineLearningServices/workspaces/project-57r5eue6ypi6s/connections/ProductCatalogAPI"
place_order_api_connection_id = "/subscriptions/29e274a2-ee66-4823-8b20-63fafc234826/resourceGroups/lab-ai-agent-service/providers/Microsoft.MachineLearningServices/workspaces/project-57r5eue6ypi6s/connections/PlaceOrderAPI"
weather_api_connection_id = "/subscriptions/29e274a2-ee66-4823-8b20-63fafc234826/resourceGroups/lab-ai-agent-service/providers/Microsoft.MachineLearningServices/workspaces/project-57r5eue6ypi6s/connections/WeatherAPI"
openai_deployment_name = "gpt-4o"

# configure_azure_monitor(connection_string=application_insights_connection_string)

with open("./product-catalog-openapi.json", "r") as f:
    openapi_product_catalog = jsonref.loads(f.read().replace("https://replace-me.local/catalogservice", f"{apim_resource_gateway_url}/catalogservice"))
openapi_tools = OpenApiTool(name="get_product_catalog", spec=openapi_product_catalog, description="Retrieve the list of products available in the catalog", 
    auth=OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=product_catalog_api_connection_id)))

with open("./place-order-openapi.json", "r") as f:
    openapi_place_order = jsonref.loads(f.read().replace("https://replace-me.local/orderservice", f"{apim_resource_gateway_url}/orderservice"))
openapi_tools.add_definition(name="place_order", spec=openapi_place_order, description="Place a product order", 
    auth=OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=place_order_api_connection_id)))

# weather 
with open("./city-weather-openapi.json", "r") as f:
    openapi_weather = jsonref.loads(f.read().replace("https://replace-me.local/weatherservice", f"{apim_resource_gateway_url}/weatherservice"))
openapi_tools.add_definition(name="get_weather", spec=openapi_weather, description="Retrieve weather information for a location", 
    auth=OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=weather_api_connection_id)))

# bing search
bing_connection = project_client.connections.get(connection_name="BingSearch")
conn_id = bing_connection.id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with OpenApi tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(model=openai_deployment_name,
        name="my-assistant",
        instructions="You are a helpful assistant that helps users order products, weather queries and Bing search. Recover from errors if any and place multiple orders if needed.",
        tools=openapi_tools.definitions + bing.definitions)
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(thread_id=thread.id,
        role="user",
        content=prompt_content)
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Print steps and function/tool details
    run_steps = project_client.agents.list_run_steps(thread_id=thread.id, run_id=run.id)
    for step in reversed(run_steps.data):
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])
        if tool_calls:
            for call in tool_calls:
                function_details = call.get("function", {})
                if function_details:
                    print(f"Function details: {function_details}")

    project_client.agents.delete_agent(agent.id)

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"🗨️ {messages.data[0].content[0].text.value}")

Created agent, ID: asst_jNN34fwTMXDtbMko4FdhZBX5
Created thread, ID: thread_4xmtBVt2d3ZNddMoCM1t44x1
Created message, ID: msg_E2dSubyrQGMPetDDsVArvUKH
Run finished with status: RunStatus.COMPLETED
Step step_Lzj9piI2DY5WwZ6HOSxr8UXM status: completed
Function details: {'name': 'get_product_catalog_get-product-details', 'arguments': '{"category":"smartphones"}', 'output': '{\r\n  "name": "N/A",\r\n  "category": "smartphones",\r\n  "sku": "SKU-4321",\r\n  "stock": 64,\r\n  "store_location": "Seattle"\r\n}'}
Step step_ycIw7ZYWNYKVnnYdxbtOhcf4 status: completed
Function details: {'name': 'place_order_PlaceOrder-invoke', 'arguments': '{"sku":"SKU-4321","quantity":11}', 'output': "{'status': 'The order was not placed because the quantity exceeds the maximum limit of five items.'}"}
Step step_ncSgzepnQfMWFukHV8m7PUZi status: completed
Step step_6y2Qc76eG0CevQ0ueTIx0yby status: completed
Function details: {'name': 'place_order_PlaceOrder-invoke', 'arguments': '{"sku":"SKU-4321","quantity":5}', 